In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from setproctitle import setproctitle
setproctitle("ENRICO_MCTS")

In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2024-08-21 09:22:00.087059: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 09:22:00.918897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-21 09:22:02.505259: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-21 09:22:02.557070: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
from tqdm import trange
from UltimateTicTacToeEnvSelfPlay import UltimateTicTacToeEnvSelfPlay
from MCTS import MCTS

In [7]:
# In this case we are making MCTS play against NUM_OF_GAMES random agents

NUM_OF_GAMES = 10
performance_envs = [UltimateTicTacToeEnvSelfPlay() for _ in range(NUM_OF_GAMES)]
dones = np.zeros((NUM_OF_GAMES,), dtype=bool)
final_rewards = np.zeros((NUM_OF_GAMES,))
while(not np.all(dones) != 0):
    mcts_agents = []
    for i in range(NUM_OF_GAMES):
        mcts_agents.append(MCTS(performance_envs[i], simulations=30000))
    print(np.count_nonzero(dones), " out of ", NUM_OF_GAMES)
    performance_states = np.array([env.to_state()[0] for env in performance_envs])
    performance_available_actions = np.array([env.to_state()[1] for env in performance_envs])
    performance_actions = []
    for i in range(NUM_OF_GAMES):
        performance_actions.append(mcts_agents[i].play())
    performance_r1 = np.zeros(NUM_OF_GAMES)
    game_finished = np.zeros(NUM_OF_GAMES)
    for i in range(NUM_OF_GAMES):
        if dones[i] == True:
            continue
        _, performance_r1[i], game_finished[i], _  = performance_envs[i].step(performance_actions[i])
        if game_finished[i] == True:
            dones[i] = True
            final_rewards[i] = performance_r1[i]

    states_opponent = np.array([env.to_state()[0] for env in performance_envs])
    available_actions_opponent = np.array([env.to_state()[1] for env in performance_envs])
    actions_opponent = np.full(available_actions_opponent.shape[0], -1, dtype=int)
    one_indices = available_actions_opponent == 1
    # For each row where there is at least one '1', select a random index of '1'
    for i in range(available_actions_opponent.shape[0]):
        valid_indices = np.where(one_indices[i])[0]
        if valid_indices.size > 0:
            actions_opponent[i] = np.random.choice(valid_indices)
    performance_r2 = np.zeros(NUM_OF_GAMES)
    game_finished = np.zeros(NUM_OF_GAMES)
    for i in range(NUM_OF_GAMES):
            if dones[i] == True:
                continue
            _, performance_r2[i], game_finished[i], _  = performance_envs[i].step(actions_opponent[i])
            # print("OPPONENT ACTION: ", actions_opponent[i])
            if game_finished[i] == True:
                dones[i] = True
                final_rewards[i] = -performance_r2[i]

win = np.count_nonzero(final_rewards == 1)/len(dones)
draw = np.count_nonzero(final_rewards == 0)/len(dones)

print(win+draw)

0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
0  out of  10
1  out of  10
1  out of  10
3  out of  10
3  out of  10
5  out of  10
6  out of  10
6  out of  10
7  out of  10
8  out of  10
8  out of  10
0.6


In [19]:
print(performance_envs[i].last_move)

None
